## Loading functions

here we export the stimdf's needed in order to run drex matlab scripts

In [1]:
# import h5py loading function for big matlab file loading
import h5py

In [2]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn import datasets, linear_model
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.linear_model import LogisticRegression, LinearRegression, Ridge
from sklearn.model_selection import train_test_split, KFold, StratifiedKFold
from sklearn.metrics import confusion_matrix,  classification_report, log_loss
# from sklearn.tree import DecisionTreeClassifier
# from sklearn.preprocessing import PolynomialFeatures
# from sklearn.svm import SVC
# from sklearn.ensemble import RandomForestClassifier
from scipy.stats import norm
import scipy.io
import re
import itertools

import os
from os.path import join
import contextlib
from copy import deepcopy
import imp 
import time 
import sys

import adaptation.longtrace_adaptation as longtrace_adaptation

/scratch/jorvhar/50550017.dccn-l029.dccn.nl/ipykernel_10178/2947205692.py:23: DeprecationWarning: the imp module is deprecated in favour of importlib; see the module's documentation for alternative uses
  import imp


In [3]:
## PLOTTING FUNCTIONS

# main loading
def data_load(pp,data_dir, stim_dir):
    """load mainpred mat file and stimuli matfile"""
    mat = scipy.io.loadmat(join(data_dir,
                                f'{pp}-mainpred.mat'))
    stimuli = scipy.io.loadmat(join(stim_dir, 
                                    f'{pp}_main_stims.mat'))
    return(mat, stimuli)


def stims_load(mat, stimuli):
    """using information from stimuli and pulse timing create dataframe 
    with frequency information, pulse location etc.
    note: 'volume_rel' & 'vol_abs' are the volume where this stimuli was measured
    'closest_volume_rel' & 'closest_volume_abs' are the volume which is the closest in time
    (half tr shift) - since a tr should capture information within that tr"""

    # set arrays
    freqz   = np.array([])
    timingz  = np.array([])
    timings_offsetz  = np.array([])
    runz     = np.array([])
    blockz   = np.array([])
    segmenz  = np.array([])
    centaz   = np.array([])
    centbz   = np.array([])
    probaz   = np.array([])
    probbz   = np.array([])

    for blk in np.arange(1, mat['timingz'][1].max()+1):
        # get blockidx
        idxblock = np.where(mat['timingz'][1] == blk) # where block is 1

        #get frequency presentation data for block
        frequencies = stimuli['pres_freq'][int(blk)-1, :]

        # other values
        tps = np.sum(mat['timingz'][3, idxblock] == 1) # get trials per secion

        #get timings back from mat file, substract begin time
        timings = mat['timingz'][6, idxblock]
        timings_offset = mat['timingz'][7, idxblock]
        matidx = np.where(mat['segmentz'][1] == blk)

        # append to arrays
        freqz = np.append(freqz, frequencies)
        timingz = np.append(timingz, timings)
        timings_offsetz = np.append(timings_offsetz, timings_offset)
        runz = np.append(runz, np.repeat(mat['segmentz'][0][matidx], tps))
        blockz = np.append(blockz, np.repeat(mat['segmentz'][1][matidx], tps))
        segmenz = np.append(segmenz, np.repeat(mat['segmentz'][2][matidx], tps))
        centaz = np.append(centaz, 2**np.repeat(mat['segmentz'][7][matidx], tps))   # cent freq a
        centbz = np.append(centbz, 2**np.repeat(mat['segmentz'][8][matidx], tps))  # cent freq b
        probaz = np.append(probaz, np.repeat(mat['segmentz'][5][matidx], tps))
        probbz = np.append(probbz, np.repeat(mat['segmentz'][6][matidx], tps))

    # oct variant 
    freqz_oct = np.log2(freqz)
    centaz_oct = np.log2(centaz)
    centbz_oct = np.log2(centbz)

    # put data into a dictionary and subsequentially in a dataframe
    stim_df_dict = {'frequencies': freqz,
                    'frequencies_oct': freqz_oct,
                    'timing': timingz,
                    'timing_offset': timings_offsetz,
                    'run': runz,
                    'block': blockz,
                    'segment': segmenz,
                    'center_freq_a': centaz,
                    'center_freq_b': centbz,
                    'center_freq_a_oct': centaz_oct,
                    'center_freq_b_oct': centbz_oct,
                    'probability_a': probaz,
                    'probability_b': probbz
                   }

    stim_df = pd.DataFrame(stim_df_dict)
    # Add the 'stimulus' column to df_beh
    stim_df['stimulus'] = stim_df.index + 1
    return(stim_df)


def sync_timing(df, sync_val, timingname='timing', new_timingname='timing_meg',
                              timingname_offset='timing_offset', new_timingname_offset='timing_offset_meg'):
    """use syncing value to get timings from stimpc domain into the MEG clock domain
    input df and sync value, returns adjusted dataframe"""

    # create new column in old dataframe
    df[new_timingname] = df[timingname] + sync_val
    df[new_timingname_offset] = df[timingname_offset] + sync_val
    # and return
    return(df)

def plot_design_mat(tr_df, all_freqs, pref1, tw1, pref2, tw2, runs=1):
    """plot the desing matrix for two tuning functions specified"""
    warnings.filterwarnings('ignore')
    
    # check if runs in list, else place in list
    runs = [runs] if isinstance(runs, int) else runs

    # create subplots
    fig, ax = plt.subplots(2,
                           2,
                           figsize=(13,13), gridspec_kw={'height_ratios': [2, 3]})

    # make x the full range
    x = all_freqs

    # calculate the normal function for all frequency points
    y1 = gauss(x, pref1, tw1)
    y2 = gauss(x, pref2, tw2)

    # plot the first gaussian functions
    ax[0, 0].plot(x,y1, color='darkgreen', lw=3)

    # pimp the first gaussian function
    ax[0, 0].tick_params(axis='x', which='major', labelsize=18)               # set ticksizes x 
    ax[0, 0].tick_params(axis='y', which='major', labelsize=18)               # and y
    ax[0, 0].set_ylabel(f'Activation', fontsize=18) 
    ax[0, 0].set_xlabel(f'Freq - oct', fontsize=18) 
    ax[0, 0].spines['top'].set_visible(False)
    ax[0, 0].spines['right'].set_visible(False)
    ax[0, 0].set_title(f'Freq: {2**pref1:.1f}kHz\nTW-FWHM: {tw1 * 2.354:.2f}oct', fontsize=22)

    # plot the second gaussian fucntion
    ax[0, 1].plot(x,y2, color='darkgreen', lw=3)

    # pimp the second gaussian function
    ax[0, 1].tick_params(axis='x', which='major', labelsize=18)               # and y
    ax[0, 1].set_xlabel(f'Freq - oct', fontsize=18) 
    ax[0, 1].axes.get_yaxis().set_visible(False)
    ax[0, 1].spines['top'].set_visible(False)
    ax[0, 1].spines['right'].set_visible(False)
    ax[0, 1].spines['left'].set_visible(False)
    ax[0, 1].set_title(f'Freq: {2**pref2:.1f}kHz\nTW-FWHM: {tw2 * 2.354:.2f}oct', fontsize=22)

    # from the tuning width and tuning pref get the columns of interest
    colls = get_tw_collumns(tr_df, pref1, tw1, convolved=True)
    del colls[-2]                    # remove adapted activation
    colls += ['onoff']     # add onoff

    # plot the first heatmap 
    sns.heatmap(normalize(tr_df[colls][tr_df['run'].isin(runs)]),cmap="crest", cbar=False, 
                ax=ax[1,0])

    # pimp the first heat map
    ax[1, 0].set_ylabel(f'Trial', fontsize=18) 
    ax[1, 0].tick_params(axis='x', which='major', labelsize=14)               # set ticksizes x 
    ax[1, 0].axes.yaxis.set_ticklabels([])

    # from the second tuning widt and tuning pref get the columns of interest
    colls = get_tw_collumns(tr_df, pref2, tw2, convolved=True)
    del colls[-2]                    # remove adapted activation
    colls += ['onoff']     # add onoff

    # plot the second heatmap
    sns.heatmap(normalize(tr_df[colls][tr_df['run'].isin(runs)]),cmap="crest", cbar=False, 
                ax=ax[1,1])

    # pimp the second heat map
    ax[1, 1].axes.get_yaxis().set_visible(False)
    ax[1, 1].tick_params(axis='x', which='major', labelsize=14)               # set ticksizes x 
    fig.tight_layout()

    plt.plot()
    return(ax,fig)


def data_plot(mat, stimuli, blocknr=1, octvspace=False):
    """plot important aspects of raw data 
    optionally plot specific blocknr"""
    ## PREPAIRING DATA
    # where block is 1
    idxblock = np.where(mat['timingz'][1] == blocknr) 

    #get frequency presentation data for block
    frequencies = stimuli['pres_freq'][blocknr-1, :]

    # other values
    tps = np.sum(mat['timingz'][3, idxblock] == 1) # get trials per secion

    #get timings back from mat file, substract begin time
    timings = mat['timingz'][4, idxblock] - np.min(mat['timingz'][4, idxblock]) 

    matidx = np.where(mat['segmentz'][1] == blocknr)

    centa = 2**np.repeat(mat['segmentz'][7][matidx], tps)   # cent freq a
    centb = 2**np.repeat(mat['segmentz'][8][matidx], tps)  # cent freq b
    proba = np.repeat(mat['segmentz'][5][matidx], tps)  # prob a
    probb = np.repeat(mat['segmentz'][6][matidx], tps)  # prob b
    
    ## ACTUAL PLOTTING
    # senatry check the data
    fig, ax = plt.subplots(2, 
                           1, 
                           sharex=True,
                           gridspec_kw={'height_ratios': [3, 1]}, figsize=(10,  7.5))

    # octave transform if wanted
    if octvspace: 
        frequencies = np.log2(frequencies)
        centa = np.log2(centa)
        centb = np.log2(centb)
    
    ax[0].scatter(timings, frequencies, color='darkslategrey', alpha=0.8)
    ax[0].axhline(y=centa[0], color='darkred', linestyle='-', alpha=0.5, ls='--', lw=4)
    ax[0].axhline(y=centb[0], color='darkred', linestyle='-', alpha=0.5, ls='--', lw=4)
    ax[0].tick_params(axis='x', which='major', labelsize=18)               # set ticksizes x 
    ax[0].tick_params(axis='y', which='major', labelsize=18)               # and y
    if octvspace: space = 'octaves'
    else: space = 'Hz'
    ax[0].set_ylabel(f'Frequencies - in {space}', fontsize=18) 
    
    #ax[1].plot(timings[0], proba, color='r')
    ax[1].plot(timings[0], probb, color='darkolivegreen', lw=8)
    ax[1].tick_params(axis='x', which='major', labelsize=18)               # set ticksizes x 
    ax[1].tick_params(axis='y', which='major', labelsize=18)               # and y
    ax[1].set_xlabel('Volume', fontsize=18)
    ax[1].set_ylabel('Prob - top', fontsize=18) 
    
    # pimp plot
    plt.xticks(fontsize=18)
    plt.yticks(fontsize=18)
    plt.suptitle(f'Stimuli over block {blocknr}', fontsize=26)
    plt.tight_layout()
    return(ax, fig)

def stim_plot(stim_df):
    ## ACTUAL PLOTTING
    # senatry check the data
    fig, ax = plt.subplots(2, 
                           1, 
                           sharex=True,
                           gridspec_kw={'height_ratios': [3, 1]}, figsize=(10,  7.5))

    ax[0].scatter(stim_df['timing'], stim_df['frequencies_oct'], color='darkslategrey', alpha=0.8)

    ax[0].plot(stim_df['timing'], stim_df['center_freq_a_oct'], linestyle='-', alpha=0.5, lw=4)
    ax[0].plot(stim_df['timing'], stim_df['center_freq_b_oct'], linestyle='-', alpha=0.5, lw=4)

    ax[0].tick_params(axis='x', which='major', labelsize=18)               # set ticksizes x 
    ax[0].tick_params(axis='y', which='major', labelsize=18)               # and y
    ax[0].set_ylabel(f'Frequencies (oct)', fontsize=18) 

    #ax[1].plot(timings[0], proba, color='r')
    ax[1].plot(stim_df['timing'], stim_df['probability_a'], color='darkolivegreen', lw=8)
    ax[1].tick_params(axis='x', which='major', labelsize=18)               # set ticksizes x 
    ax[1].tick_params(axis='y', which='major', labelsize=18)               # and y
    ax[1].set_xlabel('Stimulus nr.', fontsize=18)
    ax[1].set_ylabel('Prob - top', fontsize=18) 

    # pimp plot
    plt.xticks(fontsize=18)
    plt.yticks(fontsize=18)
    plt.suptitle(f'Stimuli', fontsize=26)
    plt.tight_layout()
    
    return(ax, fig)

def freqs_plot(pref_range, sharp_range):
    """given a sharpness range and a prefference range plot all gaussians"""

    fig, ax = plt.subplots(1, 
                           1, 
                           sharex=True, figsize=(12,  3))

    # predefine size of design matrix
    y_data = np.zeros((len(pref_range), len(pref_range)*len(sharp_range)))
    idx = 0
    
    # loop over tuning widths
    for tw in sharp_range:
        # loop over prefferences
        for pref in pref_range:
            y_data[:, idx] = gauss(pref_range, pref, tw)
            idx += 1
    plt.suptitle('Used Tuning Gaussians', fontsize=26)
    plt.tight_layout()
    plt.imshow(y_data)
    return(ax, fig)

def gauss(x, x0, sigma):
    return np.exp(-(x - x0) ** 2 / (2 * sigma ** 2))


def run_adaptation(stim_df, pref_range, sharp_range, y_decay):
    """wrapper functions to run adaptation model and return long matrixes of [pref*tw, stimuli]"""
    
    # calculate raw activation
    stims = stim_df['frequencies_oct'].to_numpy()
    activations = longtrace_adaptation.md_gaussian_activations(pref_range, sharp_range, stims)
    adaptations = np.zeros([len(pref_range)*len(sharp_range), len(stims)])
    n_back_adaptations = np.zeros([len(pref_range)*len(sharp_range), len(stims), len(y_decay)])

    for blk in stim_df['block'].unique():
        # get all stimuli within this block & get start and end idx of block
        stims = stim_df['frequencies_oct'][stim_df['block'] == blk].to_numpy()
        st_idx = stim_df.index[stim_df['block'] == blk][0]
        nd_idx = stim_df.index[stim_df['block'] == blk][-1] + 1

        # calculate adaptation for current block
        adaptations[:, st_idx:nd_idx], n_back_adaptations[:, st_idx:nd_idx, :] = longtrace_adaptation.md_stim_adaptation(stims, 
                                                                                                    y_decay, 
                                                                                                    pref_range, 
                                                                                                    sharp_range)

    # calculate adaptation weighted activations
    adapted_activations = np.multiply(adaptations, activations)

    return(activations, adaptations, adapted_activations, n_back_adaptations)


def adaptation_forwardmodel(stim_df, pref_range, sharp_range, adaptations, adapted_activations):
    """forward model the adaptation, given the stimulus presented get the experienced amount of adaptation, activation etc.
    input: stim_df: stimulus pandas dataframe with column ['frequencies_oct']
           pref_range: range of prefferences - must match adapations grid
           sharp_range: range of sharpnesses  - must match adapations grid
           adapations: adaptations grid
           adapted_activations: adapated activations grid
    returns:
           stim_df, with columns 'forward_adapation' and 'forward_adapted_activation'
           """

    # -- prepair grid
    # create a list of all indexes
    all_idxs = np.arange(len(pref_range) * len(sharp_range))

    # get grid of tuning pref and tuning sharpnesses
    tunprefs, tunsharps = longtrace_adaptation.md_get_tuning(all_idxs, pref_range, sharp_range)

    # -- tuning prefference grid possitioning
    # get minimum indx
    min_idx = np.argmin(np.abs(stim_df['frequencies_oct'].to_numpy()[:, np.newaxis] - pref_range), axis=1)

    # get discrete closest value back
    closest_val = pref_range[min_idx]

    # from discrete get boolean array
    pref_idx = closest_val == tunprefs[:, np.newaxis]

    # -- tuning width positioning
    # take one tw for now
    what_tw = sharp_range[5]

    # get bool array of where tw matches sellected tw
    tw_idx = np.tile(tunsharps == what_tw, (len(stim_df['frequencies_oct']), 1)).T

    # -- get logical intersection of the two
    # combine the two
    grid_idx = np.logical_and(pref_idx, tw_idx)
    grid_idx = np.argmax(grid_idx, axis=0)
    
    # save in dataframe
    stim_df['forward_adapation'] = adaptations[grid_idx, np.arange(len(grid_idx))]
    stim_df['forward_adapted_activation'] = adapted_activations[grid_idx, np.arange(len(grid_idx))]
    
    return(stim_df)


#drex
def stims_export_mat(pp, input_dir, stim_df, pref_range):
    """export dataframe into mat file"""
    stim_mat = {}

    # get stimuli data
    stim_mat['stims'] = stim_df.to_dict('list')

    # aditionally get range data
    stim_mat['oct_range'] = list(pref_range)
    stim_mat['freq_range'] = list(2 ** pref_range)

    scipy.io.savemat(join(input_dir, '{}_stimdf.mat'.format(pp, pp)), stim_mat)
    return


def stims_add_drex(pp, input_dir, stim_df):
    """load drex output mat, and append to dataframe"""
    # load drex mat
    mat = scipy.io.loadmat(join(input_dir,'{}_drexdf.mat'.format(pp, pp)))

    # loop over frequencies
    collumn_names = ['pred_prob_{:.3f}'.format(frq) for frq in mat['s_range'][0]]
    temp_df = pd.DataFrame(columns=collumn_names)
    for frq in range(len(mat['s_range'][0])):
        cur_frq = mat['s_range'][0, frq]
        temp_df['pred_prob_{:.3f}'.format(cur_frq)] = mat['prob_array'][frq]

    # append surprisal and predictive probabilities
    stim_df['surprisal'] = mat['surp_array'][0]
    stim_df = pd.concat([stim_df, temp_df], axis=1)
    return(stim_df)

def stims_drex_forwardmodel(pp, input_dir, stim_df, drexfn_suffix='_drexdf.mat'):
    """forward model the adaptation, given the stimulus presented get the experienced amount of adaptation, activation etc.
    input: pp: participant number
           input_dir: what folder the drexdf.mat file is located
           stim_df: stimulus pandas dataframe with column ['frequencies_oct']
           drexfn_suffix(optional): suffix of the drexfilename
    returns:
           stim_df, with columns 'surprisal' and 'pred_prob'
           """
    # load drex mat
    mat = scipy.io.loadmat(join(input_dir,'{}{}'.format(pp, drexfn_suffix)))
    tunprefs = mat['s_range'][0]

    # get minimum indx
    min_idx = np.argmin(np.abs(stim_df['frequencies_oct'].to_numpy()[:, np.newaxis] - tunprefs), axis=1)

    # add surprisal and pred prob
    stim_df['surprisal']  = mat['surp_array'][0]
    stim_df['pred_prob']  = mat['prob_array'][min_idx, np.arange(len(min_idx))]

    return(stim_df)

# loading log data

In [9]:
# fix 1 and 2, why are they wierd?

ppz = [1, 2]

# settings
task = 0     # 0: main, 1: localizer
loc_run = 1  # run 1 or run 2

for pp in ppz:

    # set behevioural directiories
    stim_dir = f'/project/3018063.01/beh/stimuli/{pp}'
    loud_dir = f'/project/3018063.01/beh/loudness/{pp}'
    data_dir = f'/project/3018063.01/beh/data/{pp}'

    #fns
    sync_fn = f'MEG_sync_sub-{pp:03d}.mat'

    # load sync files
    sync_mat = scipy.io.loadmat(join(data_dir, sync_fn))
    sync_val = sync_mat['MEG_sync']['mn'][0][0][0,0]
    
    
    ## LOAD ACTUAL LOG DATA
    
    # get mat and stimuli struct
    mat, stimuli = data_load(pp, data_dir, stim_dir)

    # put in dataframe
    df_beh = stims_load(mat, stimuli)
    df_beh = sync_timing(df_beh, sync_val)
    
    
    ## GAUSSIANS
    
    # settings
    mat = scipy.io.loadmat(join(data_dir,'{}_settings_localizer.mat'.format(pp, pp)))
    all_freqs = mat['cfg']['freq_array'][0][0][0,:]

    # must match the tonotopy settings
    tunsteps       = 10
    freqstep       = 1
    subsample      = np.arange(0,len(all_freqs), freqstep)
    mustep         = np.diff(all_freqs[subsample])
    muarray_bins   = all_freqs
    muarray        = all_freqs[subsample]

    # tuning sizes
    fwhm           = np.linspace(1,(8), tunsteps)
    octgrid        = fwhm / (2*np.sqrt(2*np.log(2)))
    sigmagrid      = 2**octgrid

    # set range of tunings and sharpnesses
    pref_range  = muarray                # match t
    sharp_range_fwhm = fwhm
    sharp_range = octgrid                # check if correct, fwhm or sigma of gaussian?

    
    ## ALREADY ADD ADAPTATION
    x_decay, y_decay = longtrace_adaptation.double_exp_decay_func(0.1399, 0.85, 0.0345, 6.82, [1, 10], 1)

    activations, adaptations, adapted_activations, n_back_adaptations = run_adaptation(df_beh, 
                                                                                               pref_range, 
                                                                                               sharp_range, 
                                                                                               y_decay)
    df_beh = adaptation_forwardmodel(df_beh, pref_range, sharp_range, adaptations, adapted_activations)

    # cleanup collumns
    del activations, adaptations, adapted_activations, n_back_adaptations
    
    
    
    ## EXPORT MATFILE FOR DREX
    stims_export_mat(pp, data_dir, df_beh, pref_range)